In [3]:
import requests
import pandas as pd
import json

In [4]:
API_KEY = "bb2c9611a7addc2bb78c7c3433c40171-097edf996c282e83518e13ad2f045839"
ACCOUNT_ID = "101-002-27231304-001"
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'


In [5]:
# Request has a session object, help maintain persisteng connection
session = requests.Session()

In [6]:
session.headers.update({
    "Authorization" :f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [7]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [8]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [9]:
response = session.get(url, params= params, data = None, headers=None)

In [10]:
response.status_code

200

In [11]:
data= response.json()
instruments_list = data['instruments']

In [12]:
len(instruments_list)

123

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [15]:
instrument_dict= {}
for i in instruments_list:
    key = i['name']
    instrument_dict[key] = {k : i[k] for k in key_i}

In [16]:
instrument_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [17]:
with open('../data/instruments.json', 'w') as f:
    f.write(json.dumps(instrument_dict, indent=2))

In [18]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data = None, headers= None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
            

    return response.status_code, data



In [19]:
code, data = fetch_candles("EUR_USD", count = 20 )

In [22]:
data[0]

{'complete': True,
 'volume': 1068,
 'time': '2023-11-16T04:00:00.000000000Z',
 'bid': {'o': '1.08321', 'h': '1.08367', 'l': '1.08314', 'c': '1.08365'},
 'mid': {'o': '1.08328', 'h': '1.08374', 'l': '1.08322', 'c': '1.08372'},
 'ask': {'o': '1.08335', 'h': '1.08382', 'l': '1.08329', 'c': '1.08380'}}

In [34]:
# time
# volume
# only want complete candles
final_data = []

for candle in data:
    if candle['complete'] ==False:
        continue
    new_dict = {}
    new_dict['time'] = candle['time']
    new_dict['volume'] = candle['volume']
    final_data.append(new_dict)
df = pd.DataFrame.from_dict(final_data)


In [35]:
df

,time,volume
0,2023-11-16T04:00:00.000000000Z,1068
1,2023-11-16T05:00:00.000000000Z,1434
2,2023-11-16T06:00:00.000000000Z,2138
3,2023-11-16T07:00:00.000000000Z,3248
4,2023-11-16T08:00:00.000000000Z,5052
5,2023-11-16T09:00:00.000000000Z,3685
6,2023-11-16T10:00:00.000000000Z,3306
7,2023-11-16T11:00:00.000000000Z,2609
8,2023-11-16T12:00:00.000000000Z,3602
9,2023-11-16T13:00:00.000000000Z,8148
